In [10]:
from gensim.models import Word2Vec

In [11]:
# IDEA: train Word2Vec on part of corpus for which we have labeled disease names
#       for new data, ask Word2Vec to predict words based on surrounding context, and if
#       it predicts a word from our labeled disease names, predict the word as a disease

In [12]:
# Some things still needed:
#       - Correctly identifying training set
#       - Creating a test set

In [13]:
# A few things to try:
#       - Stemming vs no stemming
#       - Stopword removal vs no stopword removal

In [14]:
# Simple test run for all cells below

# define training data
sentences = [['this', 'is', 'the', 'first', 'sentence', 'for', 'word2vec'],
			['this', 'is', 'the', 'second', 'sentence'],
			['yet', 'another', 'sentence'],
			['one', 'more', 'sentence'],
			['and', 'the', 'final', 'sentence']]

In [15]:
# train model
model = Word2Vec(sentences, min_count=1)

In [16]:
# summarize the loaded model
print(model)

Word2Vec(vocab=14, size=100, alpha=0.025)


In [17]:
# summarize vocabulary
words = list(model.wv.vocab)
print(words)

['this', 'is', 'the', 'first', 'sentence', 'for', 'word2vec', 'second', 'yet', 'another', 'one', 'more', 'and', 'final']


In [18]:
# access vector for one word
print(model['sentence'])

[ 1.1979045e-03 -2.4617594e-03  3.0379428e-03 -4.3931515e-03
  7.5367827e-04 -3.7958794e-03 -4.9079079e-03  3.7868472e-03
 -2.2054838e-03 -4.5656911e-03  4.0909084e-03  2.1071574e-03
  5.5827724e-04  2.2710823e-03  3.2477827e-05  2.2330747e-03
 -4.2626471e-03  1.0090796e-03  6.9845293e-04  4.3576332e-03
 -3.0159871e-03  1.8375698e-03  2.9731316e-03 -1.9725037e-03
 -7.7257515e-05  4.2118509e-03  3.0646182e-04  6.0470909e-04
  7.9023576e-04 -2.9032212e-03  4.0991791e-03  1.4218513e-03
  1.4487088e-04  2.6319786e-03 -4.0425020e-03 -1.7434331e-03
 -4.8055542e-03 -4.2288294e-03  4.7997488e-03  3.0888920e-03
 -1.6619776e-03  4.3058479e-03  2.4022744e-03 -1.1252671e-03
 -1.9903274e-03  4.0758299e-03 -9.5029641e-04  1.3462013e-03
 -4.2339447e-03  7.3138828e-04  4.1514011e-03  3.2759700e-03
  2.3173508e-03  7.5067568e-04 -2.4641834e-03 -3.4370651e-03
  1.2917391e-03  2.0453731e-04 -4.7831642e-03 -2.5302621e-03
 -1.8773470e-03  2.8503996e-03  4.2073466e-03 -2.2647227e-03
  3.0515734e-03  9.81546

/Users/tliu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [19]:
# save model
model.save('model.bin')
# load model
new_model = Word2Vec.load('model.bin')
print(new_model)

Word2Vec(vocab=14, size=100, alpha=0.025)
